In [1]:
import nibabel as nib
import matplotlib.pyplot as plt

import matplotlib.tight_bbox as tight_bbox
import matplotlib.animation as animation
import matplotlib.pylab as plt

from IPython.display import HTML
from scipy.ndimage import zoom

import numpy as np
import pandas as pd
import trimesh
from scipy.ndimage import binary_dilation, binary_erosion
import pyvista as pv

import glob
import os


def pad_to_square(image):
    H, W, S, T = image.shape
    if H == W:
        return image  # Already square​

    # Calculate padding
    if H < W:
        pad_top = (W - H) // 2
        pad_bottom = (W - H) - pad_top
        padding = ((pad_top, pad_bottom), (0, 0), (0, 0), (0, 0))
    else:  # W < H
        pad_left = (H - W) // 2
        pad_right = (H - W) - pad_left
        padding = ((0, 0), (pad_left, pad_right), (0, 0), (0, 0))

    # Apply padding
    padded_image = np.pad(image, padding, mode='constant', constant_values=0)
    return padded_image


def load_nii(nii_path):
    file = nib.load(nii_path)
    data = file.get_fdata()
    return data

def make_2d_video(image, mask, patient):
    position = image.shape[2]
    timesteps = image.shape[3]
    num_tiles = position

    grid_rows = int(np.sqrt(num_tiles) + 0.5)  # Round.
    grid_cols = (num_tiles + grid_rows - 1) // grid_rows     # Ceil.


    row_cols = np.ceil(np.sqrt(position)) 
    fig, axes = plt.subplots(grid_rows,grid_cols,figsize = (grid_cols*3,grid_rows*3))

    frames = []
    for time in range(timesteps):
        ttl = plt.text(0.5, 1.01, f'timestep = {time + 1}/{timesteps}', horizontalalignment='center', verticalalignment='bottom', transform=axes[0,0].transAxes, fontsize="large")
        artists = [ttl]
        for row, col in np.ndindex(grid_rows, grid_cols): 
            axes[row, col].axis('off')
            axes[row, col].patch.set_facecolor('white')
            pos = row * grid_cols + col
            if pos < position:
                artists.append(axes[row, col].imshow(image[:,:,pos, time], cmap = 'gray', vmin = np.min(image), vmax = np.max(image)))
                artists.append(axes[row, col].imshow(mask[:,:,pos, time, 1], alpha = mask[:,:,pos, time, 1] * 0.8, cmap = 'Blues'))
                artists.append(axes[row, col].imshow(mask[:,:,pos, time, 2], alpha = mask[:,:,pos, time, 2] * 0.8, cmap = 'jet'))
        frames.append(artists)
    bbox_inches = fig.get_tightbbox(fig.canvas.get_renderer())
    bbox_inches = bbox_inches.padded(0.1)
    tight_bbox.adjust_bbox(fig, bbox_inches)
    fig.set_size_inches(bbox_inches.width, bbox_inches.height)
    ani = animation.ArtistAnimation(fig, frames)
    ani.save(f'2dplot/{patient}.gif', fps=round(timesteps/2))
    plt.close()


# def find_crop_box(mask, crop_factor):
    '''
    Calculated a bounding box that contains the masks inside
    '''
    mask = np.eye(3)[mask] 
    mask = np.sum(mask[...,1:], axis = (2,3,4))


    x = np.sum(mask, axis = 1)
    y = np.sum(mask, axis = 0)

    top = np.min(np.nonzero(x)) - 1
    bottom = np.max(np.nonzero(x)) + 1

    left = np.min(np.nonzero(y)) - 1
    right = np.max(np.nonzero(y)) + 1
    if abs(right - left) > abs(top - bottom):
            largest_side = abs(right - left)
    else:
        largest_side = abs(top - bottom)
    x_mid = round((left + right)/2)
    y_mid = round((top + bottom)/2)
    half_largest_side = round(largest_side * crop_factor/2)
    x_max, x_min = round(x_mid + half_largest_side), round(x_mid - half_largest_side)
    y_max, y_min = round(y_mid + half_largest_side), round(y_mid - half_largest_side)
    if x_min < 0:
        x_max -= x_min 
        x_min = 0
        
    if y_min < 0:
        y_max -= y_min
        y_min = 0 
    
    return [x_min, y_min, x_max, y_max]


# Load the NIfTI file
path = 'cine_sax_compressed_derivatives'
output_path = 'clean'
patients = [pat.split('/')[-1].split('_')[-1].split('.csv')[0] for pat in glob.glob(f'{path}/*.csv')]

for patient_num, patient in enumerate(patients[:1]):
    if not os.path.exists(f'{output_path}/images/{patient}.nii.gz'):
        df = pd.read_csv(f'{path}/saxdf___{patient}.csv')

        image = load_nii(f'{path}/image___{patient}.nii.gz')
        mask = load_nii(f'{path}/masks___{patient}.nii.gz').astype('uint8')



        if df['true_slicelocation'].values[-1] < df['true_slicelocation'].values[0]:
            image = image[:,:,::-1,:]
            mask = mask[:,:,::-1,:]

        # x_min, y_min, x_max, y_max = find_crop_box(mask, crop_factor = 2)

        # image = image[y_min:y_max,x_min:x_max,...]
        # mask = mask[y_min:y_max,x_min:x_max,...]

        # image = pad_to_square(image)
        # mask = pad_to_square(mask)

        target_image = 128
        target_time = 32
        time_zoom = target_time / image.shape[3]

        # image = zoom(image, (image_zoom, image_zoom, 1, time_zoom))
        # mask = zoom(mask, (image_zoom, image_zoom, 1, time_zoom), order=0)


        # nib_image = nib.Nifti1Image(image, affine=np.eye(4))
        # nib.save(nib_image, f'{output_path}/images/{patient}.nii.gz')

        # nib_mask = nib.Nifti1Image(mask, affine=np.eye(4), dtype=np.uint8)
        # nib.save(nib_mask, f'{output_path}/masks/{patient}.nii.gz') 


        # image = load_nii(f'{output_path}/images/{patient}.nii.gz')
        # mask = load_nii(f'{output_path}/masks/{patient}.nii.gz').astype('uint8')

        # # make_2d_video(image, np.eye(3)[mask] , patient)
        # # make_3d_video(np.eye(3)[mask], patient)

In [2]:
pixel_spacings = [0.85, 1.05, 1.07]
thicknesses = [7, 8, 6]
pixel_spacing = pixel_spacings[patient_num]
thickness = thicknesses[patient_num]

In [3]:
# image = zoom(image, (pixel_spacing, pixel_spacing, thickness, time_zoom))

In [4]:
mesh_mask = zoom(mask, (pixel_spacing, pixel_spacing, thickness, time_zoom), order = 0)

In [5]:
mesh_mask = np.eye(3)[mesh_mask]

In [8]:
epi_mask = np.max(mesh_mask[...,1:], -1)

In [24]:
endo_mask = mesh_mask[...,-1]
endo_mask = binary_dilation(endo_mask, iterations=10)
endo_mask = binary_erosion(endo_mask, iterations=10)

In [34]:
myo_mask = epi_mask - endo_mask

: 

In [15]:
epi_mesh = mesh_mask.copy()
epi_mesh[epi_mesh>0] = 1
epi_mesh = trimesh.voxel.ops.matrix_to_marching_cubes(epi_mesh[...,0])
epi_mesh = trimesh.smoothing.filter_taubin(epi_mesh, iterations= 50)
epi_mesh = pv.wrap(epi_mesh).save('test.vtk')

In [16]:
myo_mesh = mesh_mask.copy()
myo_mesh[myo_mesh==1] = 1
myo_mesh[myo_mesh!=1] = 0
myo_mesh = trimesh.voxel.ops.matrix_to_marching_cubes(myo_mesh[...,0])
myo_mesh = trimesh.smoothing.filter_taubin(myo_mesh, iterations= 50)
myo_mesh = pv.wrap(myo_mesh).save('test.vtk')

In [20]:
mesh_mask.shape

(299, 299, 84, 32)

In [21]:
84/7

12.0